In [ ]:
import os
import numpy as np
import pandas as pd
import requests
import xlsxwriter 
import math
import yfinance as yf
import datetime as dt
from dotenv import load_dotenv
import requests_cache

session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
# ticker = yf.Ticker('AAPL', session=session)
# # The scraped response will be stored in the cache
# ticker.actions

# class CachedLimiterSession(
#     requests_cache.session.CacheMixin,
#     requests_ratelimiter.LimiterMixin,
#     requests.Session
#     ):
#     pass

# session = CachedLimiterSession(
#     limiter=pyrate_limiter.Limiter(
#         pyrate_limiter.RequestRate(
#             2, pyrate_limiter.Duration.SECOND*5
#             ),  # max 2 requests per 5 seconds
#     bucket_class=requests_ratelimiter.MemoryQueueBucket,
#     backend=requests_cache.SQLiteCache("yfinance.cache"),
#     )
# )

In [ ]:
def chunks(chunk_list, no_of_chunks):
    """Yield successive n-sized chunks from list"""
    for i in range(0, len(chunk_list), no_of_chunks):
        yield chunk_list[i:i + no_of_chunks]

In [ ]:
def fetch_sp500_data():
    """Fetches current financial market data from Yahoo finance public api"""
    stocks = pd.read_csv('2023-06-01-sp500.csv')
    load_dotenv()
    aapl = yf.Ticker('AAPL', session=session)
    new_columns = [*aapl.info.keys()]
    result = pd.DataFrame(columns=new_columns)
    symbol_groups = list(chunks(stocks['Ticker'], 101))
    every_str = []
    for i in range(0, len(symbol_groups)):
        every_str.append(','.join(symbol_groups[i]))
    temp = pd.DataFrame(columns=new_columns)
    for symbol_string in every_str:
        for stock in symbol_string.split(','):
            this_stock = yf.Ticker(f"{stock}", session=session)
            try:
                temp = temp.append(
                    pd.Series(
                    [
                    key for key in [*this_stock.info.values()]
                    ],
                    index = headings
                    ),
                    ignore_index=True
            )
            except(KeyError):
                pass
    return temp

In [ ]:

def convert_sp500_data_to_csv():
    """Takes current data from the Yahoo finance metrics and returns as a CSV"""
    final = fetch_sp500_data()
    today = dt.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')
    final.to_csv(f'{today}-full_sp500_data.csv')